Hi Janna,





All requried datasets can be downloaded from this link:
https://drive.google.com/drive/folders/1KPRrLJ2HUz_QdPoz3TVGwz5ewH27FcxN?usp=sharing

It contains:
- Financial news article from Reuters & Bloomberg between


Use {LOOK_BACK} last days until open of the {FORECAST} day in the future (for articles on weekends go back to friday)
Articles from NYSE start 2010-03-22 to Reuters end 2012-12-31 [not touched final test set will be 2013-01-01 to 2013-11-20 with 3901-2803=1098 articles]

### Content:
- Train text classifier on custom labels (market sematic)

#### TODO:
- Grid Testing for Parameters
- SpaCy: Find all types of entities in all news articles and store in CSV ("nlp(doc, disable=['parser', 'ner'])")
- NaiveBayes instead of LinearSVC
- Validate features by looking at most important words for each class
- Have a look into Temporal Correlation, ApEN & Cramers V, Hatemining, Related Work Text Classification
- Remove all entities before Vectorizer
- Add pretrained WordEmbedding (e.g. BERT)
- Filename_to_id for reuters and bloomberg

#### Update:
- Replace CountVectorizer mit TfidfVectorizer
- Don't trim to 200 words but by frequency (bag of words may be very large)
- Prepare Data & Write Janna

In [ ]:
import os
import re
import glob
from datetime import datetime
import sys
sys.path.append("..") # Adds higher directory to python modules path for importing from src dir

import pandas as pd
import numpy as np
import tqdm
import matplotlib
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, matthews_corrcoef, classification_report
import spacy
from spacy import displacy

%matplotlib inline
%load_ext autotime
%load_ext autoreload
%autoreload 2

### Load Data

In [ ]:
from src.datasets import NyseSecuritiesDataset
from src.datasets import NyseStocksDataset
import src.nlp_utils as nlp_utils
import src.text_classification_utils as tc_utils

HOME = ".."
DATA_DIR = "data"
REUTERS = os.path.join(HOME, DATA_DIR, "preprocessed", "news_reuters.csv")
BLOOMBERG = os.path.join(HOME, DATA_DIR, "preprocessed", "news_bloomberg.csv")
NEWS = os.path.join(HOME, DATA_DIR, "preprocessed", "news.csv")

stocks_ds = NyseStocksDataset(file_path='../data/nyse/prices-split-adjusted.csv'); stocks_ds.load()
securities_ds = NyseSecuritiesDataset(file_path='../data/nyse/securities.csv'); securities_ds.load()
companies = securities_ds.get_all_company_names()  # List[Tuple[symbol, name]]

occs_per_article = tc_utils.get_occs_per_article()

In [ ]:
LOOK_BACK = 0
FORECAST = 30
news = tc_utils.load_news_clipped(stocks_ds, LOOK_BACK, FORECAST, REUTERS)

##### Define final test run

In [ ]:
# Also contains prices from train for look back
stocks_test_ds = NyseStocksDataset(file_path='../data/nyse/prices-split-adjusted.csv', only_test=True, load=True)
all_news = pd.read_csv(NEWS, index_col=0)
# news_test = tc_utils.load_news_clipped(stocks_test_ds, look_back=0, forecast=30, file_path=REUTERS)

def final_test(pipe, look_back=0, forecast=30, epsilon_daily_label=0.01, epsilon_overall_label=0.05, min_occurrences=5):
    news_test = tc_utils.load_news_clipped(stocks_test_ds, look_back, forecast, news=all_news)
    rel_article_tuples_test = tc_utils.get_relevant_articles(
        news_test, occs_per_article, securities_ds, min_occ=min_occurrences)
    rel_article_tuples_test = [x for x in rel_article_tuples_test
                               if stocks_test_ds.is_company_available(x[0])]

    X_test = np.array([nlp_utils.get_plain_content(x[1]) for x in rel_article_tuples_test])
    y_test = tc_utils.get_discrete_labels(
        rel_article_tuples_test, stocks_test_ds, look_back=look_back, forecast=forecast,
        epsilon_daily_label=epsilon_daily_label, epsilon_overall_label=epsilon_overall_label)
    print('Test distribution:', ''.join([f'"{cls}": {sum(y_test == cls)} samples; ' for cls in [1, -1, 0]]))

    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    return acc, mcc, y_pred

### Process Data

##### Select news with enough occurrences

In [ ]:
# Get all articles with enough occurrences for one company
MIN_OCCURRENCES = 5
rel_article_tuples = tc_utils.get_relevant_articles(news, occs_per_article, securities_ds, min_occ=MIN_OCCURRENCES)
# Remove those which are not available in the training dataset of stock prices
rel_article_tuples = [x for x in rel_article_tuples if stocks_ds.is_company_available(x[0])]
print(f'Selected {len(rel_article_tuples)} relevant article tuples')

##### Generate labels

In [ ]:
EPSILON_DAILY_LABEL = 0.01
EPSILON_OVERALL_LABEL = 0.05
LOOK_BACK = 0
FORECAST = 30
labels = tc_utils.get_discrete_labels(
    rel_article_tuples, stocks_ds, look_back=LOOK_BACK, forecast=FORECAST,
    epsilon_daily_label=EPSILON_DAILY_LABEL, epsilon_overall_label=EPSILON_OVERALL_LABEL)
print(f'Generated labels for {len(labels)} articles')
print('Distribution:', ''.join([f'\n- Label "{cls}": {sum(discrete_labels == cls)} labels' for cls in [1, -1, 0]]))

##### Train pipeline

In [ ]:
X_train, y_train, X_test, y_test = tc_utils.split_shuffled(rel_article_tuples, discrete_labels, split_after_shuffle=False)
# vectorizer = CountVectorizer(tokenizer=tc_utils.tokenizeText, ngram_range=(1,1), max_features=None)
vectorizer = TfidfVectorizer(tokenizer=tc_utils.tokenizeText, ngram_range=(1,1), max_features=None)
clf = LinearSVC(max_iter=10000)  # Naive Bayesian, LogisticCLassifiert
pipe = Pipeline([('cleanText', tc_utils.CleanTextTransformer()), ('vectorizer', vectorizer), ('clf', clf)])

# train
print("Training...")
pipe.fit(X_train, y_train)
# test
print("Testing...")
y_pred = pipe.predict(X_test)
# tc_utils.inspect_vectorizer(vectorizer, clf, X_train, y_train, X_test, y_test)

In [ ]:
print(f"- Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(f"- MCC: {matthews_corrcoef(y_test, y_pred):.2f}")
print('     ', classification_report(y_test, y_pred).replace('\n', '\n      '))

# Validate on final test set

In [ ]:
test_acc, test_mcc, _ = final_test(
    pipe, look_back=LOOK_BACK, forecast=FORECAST, epsilon_daily_label=EPSILON_DAILY_LABEL,
    epsilon_overall_label=EPSILON_OVERALL_LABEL, min_occurrences=MIN_OCCURRENCES)

# Grid Tests

In [ ]:
EPSILON_DAILY_LABEL = 0.01
EPSILON_OVERALL_LABEL = 0.05
MIN_OCCURRENCES = 5  # for one company

metrics = []
pipes = []

for time_delta in tqdm([x for x in range(-90, 91, 50) if x != 0]):
    print('-'*40, '\n', f'time_delta={time_delta}')
    look_back = abs(min(time_delta, 0))
    forecast = abs(max(time_delta, 0))
    pipe, acc, mcc = tc_utils.run(
        stocks_ds, securities_ds, news, occs_per_article, time_delta=time_delta,
        epsilon_daily_label=EPSILON_DAILY_LABEL, epsilon_overall_label=EPSILON_OVERALL_LABEL,
        min_occurrences=MIN_OCCURRENCES)
    test_acc, test_mcc, _ = final_test(
        pipe, look_back=look_back, forecast=forecast, epsilon_daily_label=0.01,
        epsilon_overall_label=0.05, min_occurrences=5)
    
    metrics.append((time_delta, acc, mcc, test_acc, test_mcc))
    pipes.append(pipes)

In [ ]:
ax = pd.DataFrame(metrics, columns=['time', 'val_acc', 'val_mcc', 'test_acc', 'test_mcc']).set_index('time').plot()
ax.set_title('All News - Text Classification Metrics')
# plt.gcf().savefig('all-news-test-classification-metrics.pdf')

In [ ]:
stocks_test_ds = NyseStocksDataset(file_path='../data/nyse/prices-split-adjusted.csv', only_test=True, load=True)
news_test = pd.read_csv(NEWS, index_col=0)

pipe, acc, mcc = tc_utils.run(
    stocks_test_ds, securities_ds, news_test, occs_per_article, time_delta=30,
    epsilon_daily_label=0.01, epsilon_overall_label=0.05, min_occurrences=5)

# TODO:
- Plot with acc & val_acc for features from 50 to 5000D (https://nlp.stanford.edu/pubs/glove.pdf)
- Show misleading improvement by split_after_shuffle=True (will fail von the test set)

Tutorial: https://towardsdatascience.com/machine-learning-for-text-classification-using-spacy-in-python-b276b4051a49

### General Setting
- Use {LOOK_BACK} last days until open of the {FORECAST} day in the future (for articles on weekends go back to friday)
- Articles from NYSE start 2010-03-22 to Reuters end 2012-12-31 [not touched final test set will be 2013-01-01 to 2013-11-20 with 3901-2803=1098 articles]
- Only use title and real body (with some exceptions because of regex failure)
- TODO: Don't remove numbers, links, special characters from vectorizer

### Experiment 1
- LOOK_BACK = 30
- FORECAST = 0
- EPSILON_DAILY_LABEL = 0.01
- EPSILON_OVERALL_LABEL = 0.05
- Label "1": 829 samples, Label "-1": 1017 samples, Label "0": 957 samples
- Train: 2242 out of 2803 shuffled samples (Test: 561 samples)
- LinearSVC warns: "ConvergenceWarning: Liblinear failed to converge, increase the number of iterations."
- $val\_accuray=0.5$
- $val\_mcc=0.25$

##### Experiment 2:
- Tried calculating the mean of the relative diffs -> Values are very close to zero.
- Therefore stick to the previous method. Calculate daily label and take the mean label.

##### Experiment 3:
- LOOK_BACK=7
- Label "1": 991 labels
- Label "-1": 1106 labels
- Label "0": 706 labels
- $val\_accuray: 0.50$
- $val\_mcc: 0.23$

##### Experiment 4:
- LOOK_BACK=3
- Label "1": 834 labels, Label "-1": 920 labels, Label "0": 1049 labels
- $val\_accuray: 0.41$
- $val\_mcc: 0.11$

##### Experiment 5:
- LOOK_BACK=1
- Label "1": 620 labels,  Label "-1": 653 labels,  Label "0": 1530 labels
- $val\_accuray: 0.47$
- $val\_mcc: 0.12$

##### Experiment 6:
- LOOK_BACK=60
- Label "1": 559 labels, Label "-1": 835 labels, Label "0": 1323 labels
- $val\_accuray: 0.56$
- $val\_mcc: 0.30$

##### Experiment 7:
- LOOK_BACK=0, FORECAST=30
- Label "1": 853 labels, Label "-1": 1031 labels, Label "0": 985 labels
- $val\_accuracy: 0.56$
- $val\_mcc: 0.34$
